In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LinReg").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/13 14:33:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/13 14:33:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
from pyspark.ml.regression import LinearRegression

In [4]:
df = spark.read.format('libsvm').load("data/bodyfat.txt")
df.show()

23/02/13 14:46:37 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.
+------+--------------------+
| label|            features|
+------+--------------------+
|1.0708|(14,[0,1,2,3,4,5,...|
|1.0853|(14,[0,1,2,3,4,5,...|
|1.0414|(14,[0,1,2,3,4,5,...|
|1.0751|(14,[0,1,2,3,4,5,...|
| 1.034|(14,[0,1,2,3,4,5,...|
|1.0502|(14,[0,1,2,3,4,5,...|
|1.0549|(14,[0,1,2,3,4,5,...|
|1.0704|(14,[0,1,2,3,4,5,...|
|  1.09|(14,[0,1,2,3,4,5,...|
|1.0722|(14,[0,1,2,3,4,5,...|
| 1.083|(14,[0,1,2,3,4,5,...|
|1.0812|(14,[0,1,2,3,4,5,...|
|1.0513|(14,[0,1,2,3,4,5,...|
|1.0505|(14,[0,1,2,3,4,5,...|
|1.0484|(14,[0,1,2,3,4,5,...|
|1.0512|(14,[0,1,2,3,4,5,...|
|1.0333|(14,[0,1,2,3,4,5,...|
|1.0468|(14,[0,1,2,3,4,5,...|
|1.0622|(14,[0,1,2,3,4,5,...|
| 1.061|(14,[0,1,2,3,4,5,...|
+------+--------------------+
only showing top 20 rows



In [7]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')
lr_model = lr.fit(df)

23/02/13 14:51:50 WARN Instrumentation: [6133500b] regParam is zero, which might cause numerical instability and overfitting.
23/02/13 14:51:50 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [8]:
lr_model.coefficients

DenseVector([-0.0022, 0.0, 0.0, -0.0, 0.0, 0.0001, -0.0002, 0.0001, -0.0001, -0.0, -0.0002, -0.0002, -0.0, 0.0004])

In [9]:
lr_model.intercept

1.0981463302283172

In [10]:
train, test = df.randomSplit([0.7, 0.3])

In [11]:
lr_model = lr.fit(train)

23/02/13 14:56:17 WARN Instrumentation: [c4a23d4d] regParam is zero, which might cause numerical instability and overfitting.


In [12]:
test_results = lr_model.evaluate(test)

In [ ]:
test_results =